In [24]:
import pandas as pd
import random
import numpy as np
import random
import tensorflow as tf
import operator

In [25]:
data=pd.read_csv("../HW-2-Machine-reasoning/dataset1.csv")
data.head(10)

,x,f(x)
0,-0.66,18.37
1,2.99,5.00
2,2.65,5.12
3,-1.72,27.31
4,4.33,6.76
5,1.94,6.13
6,-1.02,21.17
7,0.05,13.73
8,-1.77,27.78
9,1.06,8.76


In [26]:
data_x = data["x"]
data_x.head(10)

0   -0.66
1    2.99
2    2.65
3   -1.72
4    4.33
5    1.94
6   -1.02
7    0.05
8   -1.77
9    1.06
Name: x, dtype: float64

In [27]:
data_y = data["f(x)"]
data_y.head(10)

0    18.37
1     5.00
2     5.12
3    27.31
4     6.76
5     6.13
6    21.17
7    13.73
8    27.78
9     8.76
Name: f(x), dtype: float64

In [28]:
F = ['+','-','*','/']

In [29]:
T = ['x']+ list(range(-5,6))

In [30]:
ran = [F,T]

In [31]:
tf = [True,False]

In [32]:
print(T)

['x', -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]


In [33]:
class Node():
    def __init__(self, *args):
        if(len(args) > 1):
            self.val = args[0]
            self.left = args[1]
            self.right = args[2]
        else:
            self.val = args[0]
            self.left = None
            self.right = None
            
    def insert_left(self, child):
        if (self.left == None):
            self.left = child
        else:
            child.left = self.left
            self.left = child
    def insert_right(self, child):
        if(self.right == None):
            self.right = child
        else:
            child.right = self.right
            self.right = child
    def evaluate(self, x):
        if(self.val in T):
            if(self.val == 'x'):
                return int(x)
            else:
                return int(self.val)
        elif(self.val in F):
            if(self.val == '+'):
                left = self.left.evaluate(x)
                right = self.right.evaluate(x)
                return left + right
            elif(self.val == '-'):
                left = self.left.evaluate(x)
                right = self.right.evaluate(x)
                return left - right
            elif(self.val == '*'):
                left = self.left.evaluate(x)
                right = self.right.evaluate(x)
                return left * right
            elif(self.val == '/'):
                if(self.right.evaluate(x) == 0):
                    return 1
                left = self.left.evaluate(x)
                right = self.right.evaluate(x)
                return left / right
    def fitness(self, x, y):
        return abs(self.evaluate(x)-y)
        

In [34]:

def print2D(root, space) :

    if (root == None) :
        return
    # Increase distance between levels
    space += 10

    print2D(root.right, space)

    # Print current node after space
    print()
    for i in range(10, space):
        print(end = " ")
    print(root.val)

    print2D(root.left, space)



In [35]:
def calculate_mse(x_data, y_data, tree):
    total = 0
    for i in range(len(x_data)):
        total += (tree.fitness(x_data[i],y_data[i]) ** 2)
    return total/len(x_data)

In [36]:
def generate_tree_shapes(size):
    root = Node(random.choice(random.choice(ran)))
    for i in range(size):
        if(random.choice(tf)):
            root.insert_right(Node(random.choice(random.choice(ran))))
            if(random.choice(tf)):
                root.right.insert_right(Node(random.choice(random.choice(ran))))
            if(random.choice(tf)):
                root.right.insert_left(Node(random.choice(random.choice(ran))))
        
        if(random.choice(tf)):
            root.insert_left(Node(random.choice(random.choice(ran))))
            if(random.choice(tf)):
                root.left.insert_right(Node(random.choice(random.choice(ran))))
            if(random.choice(tf)):
                root.left.insert_left(Node(random.choice(random.choice(ran))))

    return root

In [37]:

def generate_tree(size, root):
    if(size == 1):
        root.right = Node(random.choice(T))
        root.left = Node(random.choice(T))
    else:
        #right node
        newNode = random.choice(ran)
        if (newNode == F):
            root.right = Node(random.choice(F))
            generate_tree(size-1, root.right)
        else:
            root.right = Node(random.choice(T))
        
        #left node
        newNode = random.choice(ran)
        if (newNode == F):
            root.left = Node(random.choice(F))
            generate_tree(size-1,root.left)
        else:
            root.left = Node(random.choice(T))
root = Node(random.choice(F))

In [60]:
def mutateHelper():
    mutation = Node(random.choice(F))
    (generate_tree(3, mutation))
    return mutation

def mutate(tree):
    path = []
    path = randomSubTree(tree, path)
    mutation = mutateHelper()
    tree = replaceNode(tree, path, mutation)
    return tree


rl = [-1,1]
tf = [True, True, True, False]
def randomSubTree(node, path):
    keepGoing = random.choice(tf)
    if(keepGoing and len(path) < 3):
        nextNode = random.choice([node.right, node.left])
        if(nextNode == None):
            return path
        if(nextNode == node.right):
            path.append(1)
            randomSubTree(node.right, path)
        else:
            path.append(-1)
            randomSubTree(node.left, path)
    return path

# this function returns a subtree given a specific path
def getNode(node, path):
    if(path == []):
        return node
    if(path[0]==1):
        return getNode(node.right, path[1::])
    else:
        return getNode(node.left, path[1::])

#this function changes a node in one tree to another given node
def replaceNode(nodeA, pathA, nodeB):
    if(pathA == []):
        return nodeB
    elif(pathA[0] == 1):
        return Node(nodeA.val, nodeA.left, replaceNode(nodeA.right, pathA[1::], nodeB))
    elif(pathA[0] == -1):
        return Node(nodeA.val, replaceNode(nodeA.left, pathA[1::], nodeB), nodeA.right)

        

def crossover(treeA, treeB): 
    # determine a random node in each for crossover
    pathA = []
    pathB = []
    pathA = randomSubTree(treeA, pathA)
    pathB = randomSubTree(treeB, pathB)
    # swapping the random node in both trees
    temp = getNode(treeA, pathA)
    treeA = replaceNode(treeA, pathA, getNode(treeB, pathB))
    treeB = replaceNode(treeB, pathB, temp)
    return treeA, treeB
    
            
    

In [39]:
treeA = Node(random.choice(F))
generate_tree(3, treeA)
treeB = Node(random.choice(F))
generate_tree(3, treeB)
print("First Tree: ")
print2D(treeA,0)
print("Second Tree: ")
print2D(treeB,0)

treeA, treeB = crossover(treeA, treeB)
print("First Tree: ")
print2D(treeA,0)
print("Second Tree: ")
print2D(treeB,0)

First Tree: 

                              -2

                    *

                              -1

          *

                              x

                    *

                              -3

*

                    3

          +

                              -3

                    +

                              4
Second Tree: 

          4

-

          x
First Tree: 

                              -2

                    *

                              -1

          *

                              x

                    *

                              -3

*

          4
Second Tree: 

                    3

          +

                              -3

                    +

                              4

-

          x


In [40]:
print2D(treeB, 0)



                    3

          +

                              -3

                    +

                              4

-

          x


In [41]:
treeB = mutate(treeB)


In [42]:
print2D(treeB, 0)


                              1

                    *

                              -3

          +

                              -3

                    +

                              4

-

          x


Now we start our training

In [69]:
population = []
for i in range(10):
    tree = Node(random.choice(F))
    generate_tree(3, tree)
    population.append(tree)

epochs = 20

for i in range(epochs):
    new_generation = []
    w = []
    j = 0
    for i in population:
        w.append(calculate_mse(data_x, data_y, i))
    total = sum(w)
    print(w,min(w))
    for i in range(len(w)):
        w[i] = (total-w[i])/total
    while(j < 10):
        operator = random.choices(['r','m','c'], weights = [2,5,6], k = 1)[0]
        if(operator == 'r'):
            child = random.choices(population, weights = w, k = 1)[0]
            new_generation.append(child)
            j+=1
        elif(operator == 'm'):
            child = random.choices(population, weights = w, k = 1)[0]
            child = mutate(child)
            new_generation.append(child)
            j+=1
        elif(operator == 'c'):
            children = random.choices(population, weights = w, k = 2)
            childA = children[0]
            childB = children[1]
            crossover(childA, childB)
            new_generation.append(childA)
            new_generation.append(childB)
            j+=2
    population = []
    for i in new_generation:
        population.append(i)
    

[879.0834934319959, 999.6304734319988, 1051.3492654319914, 867.7162954320063, 1005.0881950320107, 657.8801374319916, 693.5989294319986, 750.9271174320062, 925.8022854320028, 856.4740974319976] 657.8801374319916
[1005.0881950320107, 693.5989294319986, 854.240657832006, 577.3331574319902, 856.4740974319976, 693.5989294319986, 602.7932618764576, 867.7162954320063, 812.7553054320027, 771.0365134320064] 577.3331574319902
[535.004969432, 1005.0881950320107, 693.5989294319986, 867.7162954320063, 854.240657832006, 577.3331574319902, 693.5989294319986, 854.240657832006, 577.3331574319902, 812.7553054320027, 693.5989294319986] 535.004969432
[854.240657832006, 577.3331574319902, 1005.0881950320107, 535.004969432, 592.442553431996, 760.9193154319959, 693.5989294319986, 1005.0881950320107, 854.240657832006, 577.3331574319902, 693.5989294319986] 535.004969432
[854.240657832006, 577.3331574319902, 852.0122182320063, 1013.9720814319944, 577.3331574319902, 693.5989294319986, 693.5989294319986, 854.2406

In [43]:
sum([1,2,3])

6